**THIS IS STILL IN PROGRESS**

In this notebook, I am hoping to explore more on the proper uses of Research and Statistics with Data Science. This is a Qualitative Research where I'll use Python Libraries to uncover trends and dive deeper into the problem (homicide).

DATA SCIENCE

1. Data Munging
- NaNs
- outliers
- data types

2. Data Analytics
- Using Pandas

3. Data Representation/Visualization
- Using Matplotlib, Seaborn and Plotly 

RESEARCH AND STATISTICS

 1. Describe and summarize the data
- Identify the columns used and there unique values
- Identify the mean for relevant columns
- Identify the frequency for relevant columns
- Show frequency using a graphical representation

 2. Identify relationships between variables 
 3. Compare variables 
 4. Identify the difference between variables 
 5. Forecast outcomes 

**QUESTIONS TO ANSWER AFTER ANALYSIS**

1. Has the homicide count decreased over the years?
2. Are crimes solved improving over the years?
3. What year had the most crimes? the least?
4. What state had the most crimes? the least?
5. What's the average age of a perpetrator? the victim?
6. What's the most common weapon used for homicide?
7. What's the youngest and oldest recorded age for the Victim? the Perpetrator?
8.  What's the most common relationship between the Perpetrator and Victim?
9. What month has the most crimes? is it the same every year?

In [ ]:
#import libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns

#import library to ignore warnings (just for Kaggle)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read csv data
homicide_reports = '../input/database.csv'
df = pd.read_csv(homicide_reports)

#create a new copy of the data
homicide_df = df.copy()

#drop columns that will not be used for analysis
homicide_df = (homicide_df.drop(['Record ID','Agency Code','Agency Name','Agency Type',
                                 'Victim Ethnicity','Perpetrator Ethnicity','Record Source'],axis=1))

In [ ]:
#identify number of rows and columns of the data
print(homicide_df.shape)

In [ ]:
#convert datatypes of columns to object
homicide_df[['Incident', 'Victim Count','Perpetrator Count']] = homicide_df[['Incident','Victim Count','Perpetrator Count']].astype(object)
homicide_df[['Perpetrator Age', 'Victim Count', 'Perpetrator Count']] = homicide_df[['Perpetrator Age', 'Victim Count', 'Perpetrator Count']].apply(pd.to_numeric, errors ='ignore')

#convert datatypes of columns to category
for col in ['City','State','Year','Month','Crime Type','Crime Solved','Victim Sex','Victim Race','Perpetrator Sex','Perpetrator Race','Relationship','Weapon']:
    homicide_df[col] = homicide_df[col].astype('category', ordered = True)
print(homicide_df.dtypes)
#prints categories
print(homicide_df['Victim Sex'].cat.categories)
#print numerical values
print(homicide_df['Victim Sex'].cat.codes.unique())

In [ ]:
#concise summary of the dataframe
#we don't have missing data
print(homicide_df.info())

In [ ]:
#check to see if any value is NaN in the dataframe
print('are there any null values?',homicide_df.isnull().values.any())
print('if there is string in numeric column',np.any([isinstance(val, str) for val in homicide_df['Perpetrator Age']]))

In [ ]:
#print rows with str data type from Perpetrator Age which must have int type
def check_type(homicide_df,col):
    return homicide_df.loc[homicide_df[col].apply(type)==str,col]
print('Check Type for Perpetrator Age',check_type(homicide_df, 'Perpetrator Age'))

In [ ]:
#convert data types of values other than int to NaN
homicide_df['Perpetrator Age']=pd.to_numeric(homicide_df['Perpetrator Age'], errors='coerce')
#confirm that they were converted to NaN
print('are there any null values?',homicide_df.isnull().values.any())
#convert data types of values from NaN to int
homicide_df['Perpetrator Age'] = homicide_df['Perpetrator Age'].fillna(0).astype(int)
#confirm that there are no longer NaNs
print('are there any null values?',homicide_df.isnull().values.any())
#confirm that str data types were converted to int
print('if there is string in numeric column',np.any([isinstance(val, str) for val in homicide_df['Perpetrator Age']]))

In [ ]:
#generate descriptive statistics that summarize the central tendency, dispersion 
#and shape of the dataset’s distribution, excluding NaN values.
print(homicide_df.describe())

In [ ]:
#show unique values used on the dataset
print('Crime Types = ', homicide_df['Crime Type'].unique())
print('Victime Races = ', homicide_df['Victim Race'].unique())
print('Perpetrator Races = ', homicide_df['Perpetrator Race'].unique())
print('Relationships = ', homicide_df['Relationship'].unique())
print('Weapons = ', homicide_df['Weapon'].unique())

In [ ]:
#before checking and dropping rows, check whether all indices are of unique values
print('Are indices unique?',homicide_df.index.is_unique)

In [ ]:
#check for outliers with the data

print('1980 < Year > 2014 - ',((homicide_df['Year'] > 2014) | (homicide_df['Year'] < 1980)).any())
clean_year = homicide_df[homicide_df['Year'] < 1980]
print(clean_year)

#oldest living person in USA recorded between 1980-2014 was 119 yrs old
print('Victim Age > 119 or Victim Age < 0 - ',((homicide_df['Victim Age'] > 119) | (homicide_df['Victim Age'] < 0)).any())
outlier_victim_age = homicide_df[homicide_df['Victim Age'] > 119]
print('Outliear data:\n', outlier_victim_age['Victim Age'].head())
homicide_df = homicide_df.drop(homicide_df[((homicide_df['Victim Age'] > 119) | (homicide_df['Victim Age'] < 0))].index)
print('Victim Age > 119 or Victim Age < 0 - ',((homicide_df['Victim Age'] > 119) | (homicide_df['Victim Age'] < 0)).any())
print('Perpetrator Age > 119  or Perpetrator Age < 8 - ',((homicide_df['Perpetrator Age'] > 119) | (homicide_df['Perpetrator Age'] < 8)).any())

#drop indices with Perpetrator Age < 8 since youngest recorded perpetrator for homicide between 1980-2014 was 8
homicide_df = homicide_df.drop(homicide_df[(homicide_df['Perpetrator Age'] < 8)].index)
print('Perpetrator Age < 8 - ',(homicide_df['Perpetrator Age'] < 8).any())

In [ ]:
#generate descriptive statistics that summarize the central tendency, dispersion 
#and shape of the dataset’s distribution, excluding NaN values.
print(homicide_df.describe())

In [ ]:
#count frequency of all data
cities = homicide_df['City'].value_counts().head(5)
states = homicide_df['State'].value_counts()
years = homicide_df['Year'].value_counts()
months = homicide_df['Month'].value_counts()
crime_types = homicide_df['Crime Type'].value_counts()
crime_solved = homicide_df['Crime Solved'].value_counts()
victim_sex = homicide_df['Victim Sex'].value_counts()
victim_ages = homicide_df['Victim Age'].value_counts()
victim_races = homicide_df['Victim Race'].value_counts()
perpetrator_sex = homicide_df['Perpetrator Sex'].value_counts()
perpetrator_ages = homicide_df['Perpetrator Age'].value_counts()
perpetrator_races = homicide_df['Perpetrator Race'].value_counts()
relationships = homicide_df['Relationship'].value_counts()
weapons = homicide_df['Weapon'].value_counts()

In [ ]:
#show a graphical representation for the top and bottom 5 of data
def plot_frequency(x,data):
    plt.subplot(2, 1, 1)
    sns.countplot(x=x, data=data,
                              order=data[x].value_counts().iloc[:5].index)
    plt.subplot(2, 1, 2)
    sns.countplot(x=x, data=data,
                              order=data[x].value_counts().iloc[-5:].index)        
    plt.show()

In [ ]:
plot_frequency('City',homicide_df)

In [ ]:
plot_frequency('State',homicide_df)

In [ ]:
plot_frequency('Year',homicide_df)

In [ ]:
plot_frequency('Month',homicide_df)

In [ ]:
plot_frequency('Crime Type',homicide_df)

In [ ]:
plot_frequency('Crime Solved',homicide_df)

In [ ]:
plot_frequency('Victim Sex',homicide_df)

In [ ]:
plot_frequency('Victim Age',homicide_df)

In [ ]:
plot_frequency('Victim Race',homicide_df)

In [ ]:
plot_frequency('Perpetrator Sex',homicide_df)

In [ ]:
plot_frequency('Perpetrator Age',homicide_df)

In [ ]:
plot_frequency('Perpetrator Race',homicide_df)

In [ ]:
plot_frequency('Relationship',homicide_df)

In [ ]:
plot_frequency('Weapon',homicide_df)

In [ ]:
#COMPARE VARIABLES
#show top 3 of x axis
year_crimesolved = sns.countplot(x='Year', data=homicide_df, hue='Crime Solved',
                                order=homicide_df.Year.value_counts().iloc[:3].index)

In [ ]:
def crosstab(homicide_df,row, col):
    return(pd.crosstab(homicide_df[row], homicide_df[col]))
    
page_vage = crosstab(homicide_df,'Victim Age','Perpetrator Age')
month_solved = crosstab(homicide_df,'Month','Crime Solved')
year_solved = crosstab(homicide_df,'Year','Crime Solved')
year_month = crosstab(homicide_df,'Year','Month')
relationship_vsex = crosstab(homicide_df,'Relationship','Victim Sex')
state_solved = crosstab(homicide_df,'State','Crime Solved')
relationship_weapon = crosstab(homicide_df,'Relationship','Weapon')
weapon_psex = crosstab(homicide_df,'Weapon','Perpetrator Sex')
weapon_page = crosstab(homicide_df,'Perpetrator Age','Weapon')

In [ ]:
def plot_heatmap(crosstab, annot,xtick, ytick, xrot, yrot):
    plt.figure(figsize=(8, 6))
    heatmap = sns.heatmap(crosstab,annot=annot,xticklabels=xtick, yticklabels=ytick)
    plt.xticks(rotation=xrot)
    plt.yticks(rotation=yrot)
    return heatmap

In [ ]:
plot_heatmap(page_vage,False,10,10,0,0)

In [ ]:
print(year_solved)
plot_heatmap(year_solved,True,True,True,0,0)

In [ ]:
plot_heatmap(month_solved,True,True,True,0,0)

In [ ]:
plot_heatmap(year_month,False,True,True,90,0)

In [ ]:
plot_heatmap(relationship_vsex,True,True,True,0,0)

In [ ]:
plot_heatmap(state_solved,False,True,True,0,0)

In [ ]:
plot_heatmap(relationship_weapon,False,True,True,90,0)

In [ ]:
plot_heatmap(weapon_psex,True,True,True,0,0)

In [ ]:
plot_heatmap(weapon_page,False,True,10,90,0)

**QUESTIONS TO ANSWER AFTER ANALYSIS**

1. Has the homicide count decreased over the years?
**Yes, it has. Based on the graphical representation of the top 5 years which are 1980, 1981, 1993, 1992, 1982
respectively and the least recorded were from 2000, 2014, 2013, 1999, 2011 where 2000 is the least. Also,
as you could see on the heatmap, the colors are continuously getting lighter over the years.**

2. Are crimes solved improving over the years?
**Given the crosstab and the heatmap, we could say that the percentage of crimes solved over the years is slightly decreasing**

3. What year had the most crimes? the least?
** The top 5 years which are 1980, 1981, 1993, 1992, 1982 respectively and the least recorded were from 2000, 2014, 2013, 1999, 2011 where 2000 is the least.**

4. What state had the most crimes? the least?
**for the top states, California, Texas, Florida, New York, Michigan respectively and North Dakota, Vermont, South Dakota, New Hampshire and Montana where North Dakota had the least recorded over the years.**

5. What's the average age of a perpetrator? the victim?
**Victim = 33, Perpetrator = 30**

6. What's the most common weapon used for homicide?
**Handgun**

7. What's the youngest and oldest recorded age for the Victim? the Perpetrator?
**Victim = 0, Perpetrator = 8**

8.  What's the most common relationship between the Perpetrator-Victim?
**Stranger or Acquaintance**

9. What month has the most crimes? is it the same every year?
**August/July**